# **Helmet Detection using YoloV5**

## **1. Installing dependencies**

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone the repository
%cd yolov5
%pip install -r requirements.txt # install the dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 16383, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 16383 (delta 138), reused 154 (delta 71), pack-reused 16106
Receiving objects: 100% (16383/16383), 15.09 MiB | 16.83 MiB/s, done.
Resolving deltas: 100% (11176/11176), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.1/701.1 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following d

## **2. Mounting Google Drive**
It will be used for saving the model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **3. Importing Libraries**


In [ ]:
import urllib.request
import zipfile
import os
import shutil as sh
import pandas as pd
import numpy as np

## **4. Collecting the dataset**
Download the [dataset](https://osf.io/4pwj8/)

In [ ]:
os.chdir('/content/') # Change the current working directory

In [ ]:
# Function for downloading files

def download_url(url, save_path):
    with urllib.request.urlopen(url) as dl_file:
        with open(save_path, 'wb') as out_file:
            out_file.write(dl_file.read())

In [ ]:
# Enter the active links inside the url list
url = ["",
       ""]
filenames = [""] # add the filenames of the zip files

In [ ]:
# Download all the zip files

for i, link in enumerate(url):
  download_url(link, filenames[i])

In [ ]:
# Extract the files

for file in filenames:
  with zipfile.ZipFile(file, "r") as r:
    r.extractall("/content/Images/") # Images folder will be created

Upload ***Annotations.zip***

In [ ]:
os.chdir('/content/')
with zipfile.ZipFile('Annotations.zip', 'r') as r:
  r.extractall('/content/Annotations/')

## **5. Data Preprocessing**

### Renaming the images

In [ ]:
os.chdir("/content/")
print("Starting the image renaming process")
if not os.path.exists("All"):
    os.mkdir("All")
os.chdir("./Images")
for parts in os.listdir():
    os.chdir(os.path.join(parts))
    for directory in os.listdir():
        print(f"Renaming {directory}")
        os.chdir(os.path.join(directory))
        for image in os.listdir():
            new_name = f"{directory}_{image}"
            os.rename(image, new_name)
            sh.move(new_name, "../../../All/")
        # print(f"{directory} renamed")  # Uncomment this line to see the operations
        os.chdir("../")
    os.chdir("../")
os.chdir("../")
print("All images are renamed :)")

Starting the image renaming process
Renaming Bago_urban_31
Bago_urban_31 renamed
Renaming Bago_rural_35
Bago_rural_35 renamed
Renaming Bago_highway_24
Bago_highway_24 renamed
Renaming Bago_urban_27
Bago_urban_27 renamed
Renaming Bago_highway_3
Bago_highway_3 renamed
Renaming Bago_rural_52
Bago_rural_52 renamed
Renaming Bago_highway_11
Bago_highway_11 renamed
Renaming Bago_highway_17
Bago_highway_17 renamed
Renaming Bago_urban_14
Bago_urban_14 renamed
Renaming Bago_urban_32
Bago_urban_32 renamed
Renaming Bago_highway_12
Bago_highway_12 renamed
Renaming Bago_highway_16
Bago_highway_16 renamed
Renaming Bago_rural_10
Bago_rural_10 renamed
Renaming Bago_rural_33
Bago_rural_33 renamed
Renaming Bago_urban_6
Bago_urban_6 renamed
Renaming Bago_rural_9
Bago_rural_9 renamed
Renaming Bago_rural_55
Bago_rural_55 renamed
Renaming Bago_highway_4
Bago_highway_4 renamed
Renaming Bago_rural_40
Bago_rural_40 renamed
Renaming Bago_urban_39
Bago_urban_39 renamed
Renaming Bago_rural_62
Bago_rural_62 renamed

### Creating Annotations

In [ ]:
os.chdir("/content/")
# These are the classes present inside the dataset
classes = {
    "DNoHelmetP1NoHelmetP2NoHelmet": 0,
    "DNoHelmet": 0,
    "DHelmet": 1,
    "DHelmetP1Helmet": 1,
    "DNoHelmetP0NoHelmetP1NoHelmet": 0,
    "DNoHelmetP1NoHelmet": 0,
    "DNoHelmetP1Helmet": 0,
    "DHelmetP1NoHelmetP2Helmet": 1,
    "DHelmetP1NoHelmet": 1,
    "DHelmetP1NoHelmetP2NoHelmet": 1,
    "DHelmetP0NoHelmetP1NoHelmetP2Helmet": 1,
    "DHelmetP0NoHelmetP1NoHelmet": 1,
    "DNoHelmetP0NoHelmetP1NoHelmetP2NoHelmet": 0,
    "DHelmetP0NoHelmet": 1,
    "DNoHelmetP0NoHelmet": 0,
    "DHelmetP1HelmetP2Helmet": 1,
    "DNoHelmetP1HelmetP2Helmet": 0,
    "DHelmetP0NoHelmetP1Helmet": 1,
    "DHelmetP0NoHelmetP1NoHelmetP2NoHelmet": 1,
    "DHelmetP0HelmetP1NoHelmetP2Helmet": 1,
    "DHelmetP0HelmetP1HelmetP2Helmet": 1,
    "DHelmetP0HelmetP1Helmet": 1,
    "DHelmetP1HelmetP2NoHelmet": 1,
    "DNoHelmetP1NoHelmetP2NoHelmetP3NoHelmet": 0,
    "DHelmetP0Helmet": 1,
    "DHelmetP0HelmetP1NoHelmetP2NoHelmet": 1,
    "DNoHelmetP0NoHelmetP1Helmet": 0,
    "DHelmetP0NoHelmetP1NoHelmetP2NoHelmetP3NoHelmet": 1,
    "DNoHelmetP1NoHelmetP2Helmet": 0,
    "DHelmetP1NoHelmetP2NoHelmetP3NoHelmet": 1,
    "DHelmetP1NoHelmetP2NoHelmetP3Helmet": 1,
    "DNoHelmetP0NoHelmetP1NoHelmetP2NoHelmetP3NoHelmet": 0,
    "DHelmetP0NoHelmetP1HelmetP2Helmet": 1,
    "DNoHelmetP0HelmetP1NoHelmet": 0,
    "DHelmetP0NoHelmetP1NoHelmetP2NoHelmetP3Helmet": 1,
    "DNoHelmetP0NoHelmetP1NoHelmetP2Helmet": 0,
}
print("Starting Image Annotation Process")
os.chdir("./Images")
imageFolder = []
for parts in os.listdir():
    os.chdir(os.path.join(parts))
    imageFolder.extend(os.listdir())
    os.chdir("../")
os.chdir("../")
os.chdir("./Annotations")
for dataframe in os.listdir():
    if dataframe[:-4] in imageFolder:
        # print(dataframe[:-4], "is now being processed")  # Uncomment this line to see the operations
        df = pd.read_csv(dataframe)
        for row in range(df.shape[0]):
            if df.iloc[row, 1] < 10:
                file = open(f"{dataframe[:-4]}_0{df.iloc[row, 1]}.txt", "a+")
            else:
                file = open(f"{dataframe[:-4]}_{df.iloc[row, 1]}.txt", "a+")
            file.write(
                f"{classes[df.iloc[row, 6]]} {(df.iloc[row, 2]+60)/1920:.6f} {(df.iloc[row, 3]+50)/1080:.6f} {(df.iloc[row, 4]+60)/1920:.6f} {(df.iloc[row, 5]+50)/1080:.6f}\n"
            )
            file.close()
        for text in os.listdir():
            if text[-3:] == "txt":
                sh.move(text, "../All/")
        # print(dataframe[:-4], "is completed")  # Uncomment this line to see the operations
print("All Images are annoted successfully :)")

Starting Image Annotation Process
NyaungU_urban_41 is now being processed
NyaungU_urban_41 is completed
Pathein_rural_2 is now being processed
Pathein_rural_2 is completed
Pakokku_urban_18 is now being processed
Pakokku_urban_18 is completed
Mandalay_1_128 is now being processed
Mandalay_1_128 is completed
Pakokku_urban_4 is now being processed
Pakokku_urban_4 is completed
Bago_rural_45 is now being processed
Bago_rural_45 is completed
Mandalay_1_14 is now being processed
Mandalay_1_14 is completed
Bago_rural_38 is now being processed
Bago_rural_38 is completed
NyaungU_urban_42 is now being processed
NyaungU_urban_42 is completed
Mandalay_1_88 is now being processed
Mandalay_1_88 is completed
Bago_rural_54 is now being processed
Bago_rural_54 is completed
NyaungU_urban_26 is now being processed
NyaungU_urban_26 is completed
NyaungU_urban_33 is now being processed
NyaungU_urban_33 is completed
NyaungU_urban_32 is now being processed
NyaungU_urban_32 is completed
Mandalay_1_21 is now bei

### Removing redundant images

In [ ]:

os.chdir("/content/All/")
print("Starting to remove redundant images ")
count = 0
for file in os.listdir():
    if file[-3:] == "jpg":
        temp = file[:-3] + "txt"
        if temp not in os.listdir():
            print(f"Removing {file}")
            count += 1
            os.remove(file)
print(f"{count} redundant images are removed")

Streaming output truncated to the last 5000 lines.
Removing Bago_rural_19_99.jpg
Removing Bago_highway_27_49.jpg
Removing Bago_rural_63_36.jpg
Removing Bago_rural_9_77.jpg
Removing Bago_rural_45_89.jpg
Removing Bago_highway_22_98.jpg
Removing Bago_rural_25_47.jpg
Removing Bago_rural_66_73.jpg
Removing Mandalay_1_8_25.jpg
Removing Bago_rural_25_70.jpg
Removing Bago_rural_45_81.jpg
Removing Bago_highway_14_70.jpg
Removing Bago_rural_28_09.jpg
Removing Mandalay_1_36_89.jpg
Removing Bago_urban_32_70.jpg
Removing Bago_rural_34_18.jpg
Removing Bago_urban_14_99.jpg
Removing Bago_rural_63_35.jpg
Removing Bago_rural_57_67.jpg
Removing Bago_rural_18_78.jpg
Removing Mandalay_1_106_96.jpg
Removing Bago_rural_57_76.jpg
Removing Bago_rural_24_01.jpg
Removing Bago_rural_20_67.jpg
Removing Bago_rural_26_47.jpg
Removing Bago_rural_45_40.jpg
Removing Bago_rural_38_02.jpg
Removing Bago_highway_18_95.jpg
Removing Bago_rural_25_35.jpg
Removing Bago_rural_31_34.jpg
Removing NyaungU_urban_59_74.jpg
Removing 

### Train Test Split

In [ ]:
os.chdir('/content/')
os.mkdir('Data')
os.chdir('/content/Data/')
os.mkdir('train')
os.mkdir('test')
os.mkdir('val')
os.chdir('/content/All/')

images = os.listdir()
train_size = (len(images)//2) - 200
test_size = 100
val_size = 100

np.random.shuffle(images)

c=0
for i in range(train_size):
    temp =images[0]
    if temp[-3:] == 'txt':
        sh.move(temp,'../Data/train/')
        sh.move(temp[:-3]+'jpg','../Data/train/')
        images.remove(temp[:-3]+'jpg')
        images.remove(temp)
    elif temp[-3:] == 'jpg':
        sh.move(temp,'../Data/train/')
        sh.move(temp[:-3]+'txt','../Data/train/')
        images.remove(temp[:-3]+'txt')
        images.remove(temp)
    c+=1
print("Training Images = ",c)

c=0
for i in range(test_size):
    temp =images[0]
    if temp[-3:] == 'txt':
        sh.move(temp,'../Data/test/')
        sh.move(temp[:-3]+'jpg','../Data/test/')
        images.remove(temp[:-3]+'jpg')
        images.remove(temp)
    elif temp[-3:] == 'jpg':
        sh.move(temp,'../Data/test/')
        sh.move(temp[:-3]+'txt','../Data/test/')
        images.remove(temp[:-3]+'txt')
        images.remove(temp)
    c+=1
print("Testing Images = ",c)

c =0
for i in range(val_size):
    temp =images[0]
    if temp[-3:] == 'txt':
        sh.move(temp,'../Data/val/')
        sh.move(temp[:-3]+'jpg','../Data/val/')
        images.remove(temp[:-3]+'jpg')
        images.remove(temp)
    elif temp[-3:] == 'jpg':
        sh.move(temp,'../Data/val/')
        sh.move(temp[:-3]+'txt','../Data/val/')
        images.remove(temp[:-3]+'txt')
        images.remove(temp)
    c+=1

print("Validation Images = ",c)

Training Images =  40979
Testing Images =  100
Validation Images =  100


### Creating `dataset.yml` file


```
train : ../Data/train/
test : ../Data/test/
val : ../Data/val/

nc : 2

names : ['No Helmet', 'Helmet']

```



In [ ]:
os.chdir("/content/yolov5/")

with open('dataset.yml', 'w') as f:
  f.write("train : ../Data/train/\n")
  f.write("test : ../Data/test/\n")
  f.write("val : ../Data/val/\n\n")
  f.write("nc : 2\n\n")
  f.write("names : ['No Helmet', 'Helmet']")

## **6. Model Development**

In [ ]:
!python train.py --img 340 --batch 16 --epochs 30 --data dataset.yml --weights yolov5n.pt --cache ram

2024-01-20 07:35:00.874818: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 07:35:00.874940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-20 07:35:00.993335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5n.pt, cfg=, data=dataset.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=340, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, opti

## **7. Saving the files in the drive**

In [ ]:
os.mkdir('/content/drive/MyDrive/EXP_3') # Create a folder in the drive
os.chdir("/content/")
sh.move("/content/yolov5/runs/train/exp/weights/best.pt","/content/drive/MyDrive/EXP_3/") # Change the path according to the exp
sh.move("/content/yolov5/runs/train/exp/weights/last.pt","/content/drive/MyDrive/EXP_3/")

'/content/drive/MyDrive/EXP_3/last.pt'